In [4]:
import os
import pandas as pd 

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest, GetOrderByIdRequest
from alpaca.trading.enums import OrderSide, TimeInForce, QueryOrderStatus
from dotenv import load_dotenv

class AlpacaTradingAccount: 

    def __init__(self, env_path: str = None, paper = True):
        self.env_path = env_path
        self.paper = paper
        self.api_key, self.secret_key = self._load_environment()
        self.client = TradingClient(api_key=self.api_key, secret_key=self.secret_key, paper = self.paper)

    def _load_environment(self):
        if self.env_path is not None: 
            load_dotenv(self.env_path) 

        else: 
            load_dotenv()

        # Then we are going to get the API keys
        if self.paper: 
            api_key = os.environ['ALPACA_PAPER_TRADING_KEY_ID']
            secret_key = os.environ['ALPACA_PAPER_TRADING_SECRET_KEY']
        else: 
            api_key = os.environ['ALPACA_LIVE_TRADING_KEY_ID']
            secret_key = os.environ['ALPACA_LIVE_TRADING_SECRET_KEY']

        return api_key, secret_key

    def get_all_active_positions(self) -> pd.DataFrame:
        positions = self.client.get_all_positions()
        positions_list = [] 
        for position in positions: 
            position_dict = dict(position)
            positions_list.append(dict(position))

        positions_df = pd.DataFrame(positions_list)
        as_numeric_columns = ['avg_entry_price', 'qty', 'market_value', 'cost_basis', 'unrealized_pl', 'unrealized_plpc', 'unrealized_intraday_pl', 'unrealized_intraday_plpc', 'current_price', 'lastday_price', 'change_today']
        positions_df[as_numeric_columns] = positions_df[as_numeric_columns].astype(float).round(3)

        return positions_df

    def get_all_closed_positions(self):
        pass

    def get_all_active_orders(self):
        pass

In [5]:
paper_account = AlpacaTradingAccount()

In [9]:
positions = paper_account.client.get_all_positions()

In [10]:
import pandas as pd 

In [14]:
first_position = positions[0]

How can we parse through the information in the position object

We have
* Asset class 
* asset id
* average entry price 
* change today 
* cost_basis
* exchange that this trades on 
* last price 
* market value of the positoin 
* quantity
* side that we are on 
* symbol
* unrealized pnl

In [27]:
str(first_position.asset_class)

'AssetClass.US_EQUITY'

In [30]:
first_position.asset_id

UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415')

In [31]:
first_position.avg_entry_price

'150.81'

In [33]:
first_position

{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'avg_entry_price': '150.81',
    'change_today': '-0.0095819387018442',
    'cost_basis': '150.81',
    'current_price': '150.91',
    'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
    'lastday_price': '152.37',
    'market_value': '150.91',
    'qty': '1',
    'side': <PositionSide.LONG: 'long'>,
    'symbol': 'AAPL',
    'unrealized_intraday_pl': '0.1',
    'unrealized_intraday_plpc': '0.0006630860022545',
    'unrealized_pl': '0.1',
    'unrealized_plpc': '0.0006630860022545'}

In [38]:
positions_list = [] 
for position in positions: 
    position_dict = dict(position)
    positions_list.append(dict(position))

In [39]:
positions_df = pd.DataFrame(positions_list)

In [44]:
as_numeric_columns = ['avg_entry_price', 'qty', 'market_value', 'cost_basis', 'unrealized_pl', 'unrealized_plpc', 'unrealized_intraday_pl', 'unrealized_intraday_plpc', 'current_price', 'lastday_price', 'change_today']
positions_df[as_numeric_columns] = positions_df[as_numeric_columns].astype(float).round(3)


In [46]:
positions_df

,asset_id,symbol,exchange,asset_class,avg_entry_price,qty,side,market_value,cost_basis,unrealized_pl,unrealized_plpc,unrealized_intraday_pl,unrealized_intraday_plpc,current_price,lastday_price,change_today
0,b0b6dd9d-8b9b-48a9-ba46-b9d54906e415,AAPL,AssetExchange.NASDAQ,AssetClass.US_EQUITY,150.81,1.00,PositionSide.LONG,150.91,150.81,0.10,0.001,0.10,0.001,150.91,152.37,-0.010
1,64bbff51-59d6-4b3c-9351-13ad85e3c752,BTCUSD,AssetExchange.FTXU,AssetClass.CRYPTO,19573.00,0.23,PositionSide.LONG,4554.23,4501.79,52.44,0.012,52.44,0.012,19801.00,19745.00,0.003
2,1f8bfbdf-083a-4ebe-ad36-eaf6b6f0cb9d,TLT,AssetExchange.NASDAQ,AssetClass.US_EQUITY,107.18,-100.00,PositionSide.SHORT,-10738.00,-10718.00,-20.00,-0.002,-20.00,-0.002,107.38,107.97,-0.005
